In [1]:
import pandas as pd
train_path = "../input/multilingual-fake-news/TALLIP-FakeNews-Dataset/Tamil/Train/train_Tamil_Data"
test_path = "../input/multilingual-fake-news/TALLIP-FakeNews-Dataset/Tamil/Test/test_Tamil_Data"

In [2]:
def get_df(path):
    bus = pd.read_table(path+'_Business.txt',header=0,names=('Domain','Topic','News','Label'))
    cel = pd.read_table(path+'_Celebrity_Full.txt',header=0,names=('Domain','Topic','News','Label'))
    com = pd.read_table(path+'_Complete_FakeNews.txt',header=0,names=('Domain','Topic','News','Label'))
    edu = pd.read_table(path+'_Education.txt',header=0,names=('Domain','Topic','News','Label'))
    ent = pd.read_table(path+'_Entertainment.txt',header=0,names=('Domain','Topic','News','Label'))
    pol = pd.read_table(path+'_Politics.txt',header=0,names=('Domain','Topic','News','Label'))
    spo = pd.read_table(path+'_Sports.txt',header=0,names=('Domain','Topic','News','Label'))
    tec = pd.read_table(path+'_Technology.txt',header=0,names=('Domain','Topic','News','Label'))
    data = pd.concat([bus,cel,com,edu,ent,pol,spo,tec],ignore_index=True)
    return data

In [3]:
train_data = get_df(train_path)
test_data = get_df(test_path)

In [4]:
print(train_data.isna().sum()) #23 titles null
print(test_data.isna().sum()) # no null

Domain     0
Topic     22
News       1
Label      0
dtype: int64
Domain     0
Topic     14
News       1
Label      0
dtype: int64


In [5]:
def convert_data(df):
    df['Topic_News'] = df['Topic'].astype(str)+"."+df['News'].astype(str)
    df['Label'] = df['Label'].str.upper()
    return df
train_data,test_data = convert_data(train_data),convert_data(test_data)

In [ ]:
Fake = train_data[train_data.Label=='FAKE']['Topic_News']
Legit = train_data[train_data.Label=='LEGIT']['Topic_News']

In [ ]:
import matplotlib.pyplot as plt
color = ['Blues_r', 'Reds_r',]
splitted_data = [Fake, Legit]
for item in range(len(splitted_data)):
    plt.figure(figsize=(10,5))
    pd.Series(' '.join([i for i in splitted_data[item]]).split()).value_counts().head(20).plot(kind='bar',colormap=color[item])

In [6]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
train_data['labels']=encoder.fit_transform(train_data['Label'])
test_data['labels']=encoder.fit_transform(test_data['Label'])
print(train_data['labels'].value_counts())
print(test_data['labels'].value_counts())

1    609
0    579
Name: labels, dtype: int64
0    401
1    371
Name: labels, dtype: int64


In [7]:
#!pip install - q transformers
from transformers import AutoTokenizer,AutoModel,AutoModelForSequenceClassification
MODEL_TYPE = 'xlmroberta'#'bert'
MODEL_NAME = 'xlm-roberta-base' #'bert-base-uncased'
#tokenizer =  AutoTokenizer.from_pretrained(MODEL_NAME) 

In [8]:
MODEL_TYPE = 'bert'
MODEL_NAME = 'google/muril-base-cased' #'bert-base-uncased'

In [ ]:
MODEL_TYPE = 'albert'
MODEL_NAME = 'ai4bharat/indic-bert'

In [11]:
MODEL_TYPE = 'bert'
MODEL_NAME = 'bert-base-multilingual-uncased'

In [ ]:
token_counts = []
for _, row in train_data.iterrows():
    token_count = len(tokenizer.encode(
    row["Topic_News"], 
    max_length=1024, 
    truncation=True
  ))
    token_counts.append(token_count)
import seaborn as sns
sns.histplot(token_counts)
plt.xlim([0, 750]);

In [ ]:
token_counts = []
for _, row in test_data.iterrows():
    token_count = len(tokenizer.encode(
    row["Topic_News"], 
    max_length=1024, 
    truncation=True
  ))
    token_counts.append(token_count)
import seaborn as sns
sns.histplot(token_counts)
plt.xlim([0, 750]);

In [ ]:
import matplotlib.pyplot as plt
train_data["Words Per Tweet"] = train_data["Topic_News"].str.split().apply(len)
train_data.boxplot("Words Per Tweet", by="Label", grid=False, showfliers=True,
           color="black")
plt.suptitle("")
plt.xlabel("")
plt.show()

In [9]:
!pip install -q simpletransformers
import torch
from simpletransformers.classification import ClassificationModel,ClassificationArgs
model_args = ClassificationArgs(
    num_train_epochs=5,
    overwrite_output_dir=True, 
    manual_seed=42,
    max_seq_length=512,
    #sliding_window=True,
    eval_batch_size=16,
    do_lower_case=True,
   #stride=0.9,
    early_stopping_patience=2,
    save_model_every_epoch=False,
    early_stopping_consider_epochs=True,
    use_early_stopping=True,
    evaluate_during_training=True,
    #wandb_project='ST-Hope-CELoss',
    learning_rate=3e-05)
model = ClassificationModel(
    model_type=MODEL_TYPE,#'deberta',
    model_name=MODEL_NAME, 
    use_cuda=torch.cuda.is_available(),
    #cuda_device=3,
    num_labels=len(encoder.classes_), 
    args=model_args)

Downloading:   0%|          | 0.00/411 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/909M [00:00<?, ?B/s]

Some weights of the model checkpoint at google/muril-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not in

Downloading:   0%|          | 0.00/206 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.02M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/113 [00:00<?, ?B/s]

In [10]:
train_df=pd.DataFrame()
train_df['text']=train_data['Topic_News']
train_df['labels']=train_data['labels']

In [11]:
test_df=pd.DataFrame()
test_df['text']=test_data['Topic_News']
test_df['labels']=test_data['labels']

In [12]:
from sklearn.model_selection import train_test_split
train_df, valid_df = train_test_split(train_df,test_size=0.2,random_state=42,stratify=train_df['labels'])
print(train_df.labels.value_counts())
print(valid_df.labels.value_counts())

1    487
0    463
Name: labels, dtype: int64
1    122
0    116
Name: labels, dtype: int64


In [13]:
model.train_model(train_df,eval_df=valid_df,args={'train_batch_size':16})

  0%|          | 0/950 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 0 of 5:   0%|          | 0/60 [00:00<?, ?it/s]

  0%|          | 0/238 [00:00<?, ?it/s]

Running Epoch 1 of 5:   0%|          | 0/60 [00:00<?, ?it/s]

...

Running Epoch 3 of 5:   0%|          | 0/60 [00:00<?, ?it/s]

  0%|          | 0/238 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/60 [00:00<?, ?it/s]

  0%|          | 0/238 [00:00<?, ?it/s]

(300,
 defaultdict(list,
             {'global_step': [60, 120, 180, 240, 300],
              'train_loss': [0.6925455927848816,
               0.77783203125,
               0.5372314453125,
               0.6638997197151184,
               0.37451171875],
              'mcc': [0.33530816323964335,
               0.3628040252088524,
               0.5896031561822084,
               0.6033305546350596,
               0.6921892071718996],
              'tp': [86, 112, 94, 108, 110],
              'tn': [73, 45, 95, 82, 91],
              'fp': [43, 71, 21, 34, 25],
              'fn': [36, 10, 28, 14, 12],
              'auroc': [0.7056953080836631,
               0.7526851328434143,
               0.8626695873374788,
               0.8843626342566422,
               0.8951738270209157],
              'auprc': [0.6943521170684842,
               0.7452593522085557,
               0.8687728898431308,
               0.8834759354719888,
               0.8917360320731088],
              'eva

In [14]:
valid_predictions, raw_outputs = model.predict(valid_df['text'].tolist())
from sklearn.metrics import confusion_matrix, classification_report,f1_score
print('class_report\n',classification_report(valid_df['labels'],valid_predictions,target_names=list(encoder.classes_),digits=4))

  0%|          | 0/238 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

class_report
               precision    recall  f1-score   support

        FAKE     0.8835    0.7845    0.8311       116
       LEGIT     0.8148    0.9016    0.8560       122

    accuracy                         0.8445       238
   macro avg     0.8492    0.8431    0.8435       238
weighted avg     0.8483    0.8445    0.8439       238



In [15]:
test_predictions, raw_outputs = model.predict(test_df['text'].tolist())

  0%|          | 0/772 [00:00<?, ?it/s]

  0%|          | 0/49 [00:00<?, ?it/s]

In [16]:
from sklearn.metrics import confusion_matrix, classification_report,f1_score
print('class_report\n',classification_report(test_df['labels'],test_predictions,target_names=list(encoder.classes_),digits=4))

class_report
               precision    recall  f1-score   support

        FAKE     0.8929    0.7481    0.8141       401
       LEGIT     0.7683    0.9030    0.8302       371

    accuracy                         0.8225       772
   macro avg     0.8306    0.8255    0.8222       772
weighted avg     0.8330    0.8225    0.8219       772

